PROJETO: ANÁLISE EXPLORATÓRIA DE DADOS (EDA) COM MACHINE LEARNING
Análise de Exportação com Previsões para 2026

In [32]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor
import warnings
warnings.filterwarnings('ignore')

In [33]:
# 1. LIMPEZA E PRÉ-PROCESSAMENTO DE DADOS
# Carregar dados
df_exportacao = pd.read_csv("V_EXPORTACAO_GERAL_2019-01_2024-12_DT20250428 (2).csv.csv")

In [34]:
# Limpeza básica
df_limpo = df_exportacao.copy()
df_limpo = df_limpo.dropna().drop_duplicates()

In [35]:
# Padronização de Nomes e Colunas
df_limpo.rename(columns={
    "Países": "Pais",
    "Valor US$ FOB": "Valor_USD",
    "Quilograma Líquido": "Peso_kg",
    "Descrição SH2": "Produto"
}, inplace=True)

In [36]:
# Limpeza e conversão de tipos
df_limpo["Ano"] = df_limpo["Ano"].astype(int)
df_limpo["Valor_USD"] = pd.to_numeric(df_limpo["Valor_USD"], errors='coerce')
df_limpo["Peso_kg"] = pd.to_numeric(df_limpo["Peso_kg"], errors='coerce')
df_limpo["Pais"] = df_limpo["Pais"].astype(str).str.strip().str.replace(r"[\n\r\t]", "", regex=True)
df_limpo["Produto"] = df_limpo["Produto"].astype(str).str.strip().str.replace(r"[\n\r\t]", "", regex=True)

In [37]:
# Dataframe final limpo
df = df_limpo[["Ano", "Pais", "Produto", "Valor_USD", "Peso_kg"]].dropna()

In [38]:
print(f"Dados após limpeza: {len(df)} registros")
print(f"Período: {df['Ano'].min()} - {df['Ano'].max()}")
print(f"Países únicos: {df['Pais'].nunique()}")
print(f"Produtos únicos: {df['Produto'].nunique()}")

Dados após limpeza: 5566 registros
Período: 2019 - 2024
Países únicos: 10
Produtos únicos: 97


In [39]:
# 2. ANÁLISE DESCRITIVA DOS DADOS
print("   ESTATÍSTICAS DE VALOR (US$ FOB):")
print(f"   Valor total: US$ {df['Valor_USD'].sum():,.0f}")
print(f"   Valor médio: US$ {df['Valor_USD'].mean():,.0f}")
print(f"   Valor mediano: US$ {df['Valor_USD'].median():,.0f}")
print(f"   Desvio padrão: US$ {df['Valor_USD'].std():,.0f}")

print("\n  ESTATÍSTICAS DE PESO:")
print(f"   Peso total: {df['Peso_kg'].sum():,.0f} kg")
print(f"   Peso médio: {df['Peso_kg'].mean():,.0f} kg")

   ESTATÍSTICAS DE VALOR (US$ FOB):
   Valor total: US$ 1,053,349,216,026
   Valor médio: US$ 189,247,074
   Valor mediano: US$ 5,437,202
   Desvio padrão: US$ 1,386,377,793

  ESTATÍSTICAS DE PESO:
   Peso total: 2,998,941,989,338 kg
   Peso médio: 538,796,620 kg


In [40]:
# Análise temporal
print("\n EVOLUÇÃO TEMPORAL:")
evolucao_anual = df.groupby('Ano')['Valor_USD'].sum()
for ano in sorted(df['Ano'].unique()):
    valor = evolucao_anual[ano]
    if ano > df['Ano'].min():
        crescimento = ((valor - evolucao_anual[ano-1]) / evolucao_anual[ano-1]) * 100
        print(f"   {ano}: US$ {valor:,.0f} ({crescimento:+.1f}%)")
    else:
        print(f"   {ano}: US$ {valor:,.0f}")


 EVOLUÇÃO TEMPORAL:
   2019: US$ 137,672,885,891
   2020: US$ 128,669,898,513 (-6.5%)
   2021: US$ 173,764,493,491 (+35.0%)
   2022: US$ 198,316,801,044 (+14.1%)
   2023: US$ 212,515,398,380 (+7.2%)
   2024: US$ 202,409,738,707 (-4.8%)


In [41]:
# Top análises para EDA
print("\n TOP 10 PAÍSES (VALOR TOTAL 2019-2024):")
top_paises_historico = df.groupby('Pais')['Valor_USD'].sum().nlargest(10)
for i, (pais, valor) in enumerate(top_paises_historico.items(), 1):
    print(f"   {i:2d}. {pais}: US$ {valor:,.0f}")

print("\n TOP 10 PRODUTOS (VALOR TOTAL 2019-2024):")
top_produtos_historico = df.groupby('Produto')['Valor_USD'].sum().nlargest(10)
for i, (produto, valor) in enumerate(top_produtos_historico.items(), 1):
    produto_nome = produto[:50] + "..." if len(produto) > 50 else produto
    print(f"   {i:2d}. {produto_nome}: US$ {valor:,.0f}")


 TOP 10 PAÍSES (VALOR TOTAL 2019-2024):
    1. China: US$ 507,178,095,899
    2. Estados Unidos: US$ 197,053,957,653
    3. Argentina: US$ 75,993,183,435
    4. Países Baixos (Holanda): US$ 58,976,545,576
    5. Espanha: US$ 41,108,766,584
    6. Chile: US$ 39,728,054,760
    7. México: US$ 37,712,962,704
    8. Japão: US$ 33,916,330,877
    9. Alemanha: US$ 31,660,964,957
   10. Canadá: US$ 30,020,353,581

 TOP 10 PRODUTOS (VALOR TOTAL 2019-2024):
    1. Sementes e frutos oleaginosos; grãos, sementes e f...: US$ 192,873,898,950
    2. Combustíveis minerais, óleos minerais e produtos d...: US$ 173,741,465,965
    3. Minerios, escórias e cinzas: US$ 151,511,435,227
    4. Carnes e miudezas, comestíveis: US$ 64,117,619,337
    5. Ferro fundido, ferro e aço: US$ 58,625,272,393
    6. Reatores nucleares, caldeiras, máquinas, aparelhos...: US$ 43,463,933,350
    7. Veículos automóveis, tratores, ciclos e outros veí...: US$ 40,212,968,009
    8. Pastas de madeira ou de outras matérias fibro

In [42]:
# 3. IDENTIFICAÇÃO DE VARIÁVEIS E CORRELAÇÕES
# Criar dataset agregado para análise de correlações
df_corr = df.groupby(['Ano', 'Pais', 'Produto']).agg({
    'Valor_USD': 'sum',
    'Peso_kg': 'sum'
}).reset_index()

In [43]:
# Adicionar variáveis derivadas
df_corr['Valor_por_kg'] = df_corr['Valor_USD'] / (df_corr['Peso_kg'] + 1)
df_corr['Log_Valor'] = np.log1p(df_corr['Valor_USD'])
df_corr['Log_Peso'] = np.log1p(df_corr['Peso_kg'])

In [44]:
# Matriz de correlação das variáveis numéricas
correlacoes = df_corr[['Ano', 'Valor_USD', 'Peso_kg', 'Valor_por_kg', 'Log_Valor', 'Log_Peso']].corr()

print(" MATRIZ DE CORRELAÇÕES:")
print(correlacoes.round(3))

 MATRIZ DE CORRELAÇÕES:
                Ano  Valor_USD  Peso_kg  Valor_por_kg  Log_Valor  Log_Peso
Ano           1.000      0.022    0.004        -0.008      0.037     0.005
Valor_USD     0.022      1.000    0.706        -0.002      0.257     0.252
Peso_kg       0.004      0.706    1.000        -0.003      0.135     0.151
Valor_por_kg -0.008     -0.002   -0.003         1.000     -0.009    -0.077
Log_Valor     0.037      0.257    0.135        -0.009      1.000     0.926
Log_Peso      0.005      0.252    0.151        -0.077      0.926     1.000


In [45]:
# Análise de variabilidade por país e produto
print("\n VARIABILIDADE POR PAÍS (TOP 5):")
variabilidade_pais = df.groupby('Pais')['Valor_USD'].agg(['std', 'mean']).reset_index()
variabilidade_pais['CV'] = variabilidade_pais['std'] / variabilidade_pais['mean']
variabilidade_pais = variabilidade_pais.nlargest(5, 'CV')
for _, row in variabilidade_pais.iterrows():
    print(f"   {row['Pais']}: CV = {row['CV']:.3f}")


 VARIABILIDADE POR PAÍS (TOP 5):
   Espanha: CV = 4.842
   China: CV = 4.530
   Canadá: CV = 4.147
   Chile: CV = 3.874
   Alemanha: CV = 3.385


In [46]:
# 4. PREPARAÇÃO PARA MACHINE LEARNING
# Dataset para ML
df_ml = df.groupby(['Ano', 'Pais', 'Produto']).agg({
    'Valor_USD': 'sum',
    'Peso_kg': 'sum'
}).reset_index()

In [47]:
# Features engineering
df_ml['Valor_por_kg'] = df_ml['Valor_USD'] / (df_ml['Peso_kg'] + 1)

In [48]:
# Encoding de variáveis categóricas
le_pais = LabelEncoder()
le_produto = LabelEncoder()
df_ml['Pais_encoded'] = le_pais.fit_transform(df_ml['Pais'])
df_ml['Produto_encoded'] = le_produto.fit_transform(df_ml['Produto'])

In [49]:
# Criar features de lag temporal
df_ml = df_ml.sort_values(['Pais', 'Produto', 'Ano'])
df_ml['Valor_lag1'] = df_ml.groupby(['Pais', 'Produto'])['Valor_USD'].shift(1)
df_ml['Valor_lag2'] = df_ml.groupby(['Pais', 'Produto'])['Valor_USD'].shift(2)
df_ml['Crescimento_pct'] = df_ml.groupby(['Pais', 'Produto'])['Valor_USD'].pct_change()

In [50]:
# Remover NaN gerados pelos lags
df_ml_limpo = df_ml.dropna()

print(f"Dataset ML preparado: {len(df_ml_limpo)} registros para treinamento")

Dataset ML preparado: 3655 registros para treinamento


In [51]:
# 5. TREINAMENTO DOS MODELOS ML
# Definir features e target
features = ['Ano', 'Pais_encoded', 'Produto_encoded', 'Peso_kg',
           'Valor_por_kg', 'Valor_lag1', 'Valor_lag2', 'Crescimento_pct']
X = df_ml_limpo[features]
y = df_ml_limpo['Valor_USD']

In [52]:
# Divisão treino/teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [53]:
# Modelos para testar
modelos = {
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'Linear Regression': LinearRegression(),
    'Decision Tree': DecisionTreeRegressor(random_state=42)
}

In [54]:
# Avaliar modelos
resultados_ml = {}
melhor_modelo = None
melhor_r2 = -np.inf

In [55]:
for nome, modelo in modelos.items():
    # Treinar modelo
    if nome == 'Linear Regression':
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        modelo.fit(X_train_scaled, y_train)
        y_pred = modelo.predict(X_test_scaled)
    else:
        modelo.fit(X_train, y_train)
        y_pred = modelo.predict(X_test)

In [56]:
# Métricas
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

resultados_ml[nome] = {'R2': r2, 'MAE': mae}

print(f"{nome}: R² = {r2:.4f}, MAE = US$ {mae:,.0f}")

Decision Tree: R² = 0.9763, MAE = US$ 26,523,218


In [57]:
# Guardar melhor modelo
if r2 > melhor_r2:
  melhor_r2 = r2
  melhor_modelo = modelo
  melhor_nome = nome
  if nome == 'Linear Regression':
    melhor_scaler = scaler

print(f"\n Melhor modelo: {melhor_nome} (R² = {melhor_r2:.4f})")


 Melhor modelo: Decision Tree (R² = 0.9763)


In [58]:
# 6. PREVISÕES PARA 2026
# Preparar dados base para previsão (usando 2024 como referência)
dados_2024 = df_ml_limpo[df_ml_limpo['Ano'] == 2024].copy()

In [59]:
# Criar dataset para 2026
dados_2026 = dados_2024.copy()
dados_2026['Ano'] = 2026
dados_2026['Valor_lag2'] = dados_2026['Valor_lag1']
dados_2026['Valor_lag1'] = dados_2026['Valor_USD']

In [60]:
# Estimar crescimento médio por combinação país-produto
crescimento_medio = df_ml_limpo.groupby(['Pais', 'Produto'])['Crescimento_pct'].mean().reset_index()
dados_2026 = dados_2026.merge(crescimento_medio, on=['Pais', 'Produto'], how='left', suffixes=('', '_medio'))
dados_2026['Crescimento_pct'] = dados_2026['Crescimento_pct_medio'].fillna(dados_2026['Crescimento_pct'])
dados_2026.drop('Crescimento_pct_medio', axis=1, inplace=True)

In [61]:
# Fazer previsões
X_2026 = dados_2026[features]

In [62]:
if melhor_nome == 'Linear Regression':
    X_2026_scaled = melhor_scaler.transform(X_2026)
    previsoes_2026 = melhor_modelo.predict(X_2026_scaled)
else:
    previsoes_2026 = melhor_modelo.predict(X_2026)

dados_2026['Valor_Previsto_2026'] = previsoes_2026

print(f" Previsões geradas para {len(dados_2026)} combinações país-produto")

 Previsões geradas para 923 combinações país-produto


In [63]:
# 7. DATASETS PARA LOOKER STUDIO
# Dataset 1: Dados históricos (2019-2024) - Para gráficos históricos
df_historico = df.copy()
df_historico['Tipo'] = 'Histórico'
df_historico.rename(columns={'Valor_USD': 'Valor'}, inplace=True)

In [64]:
# Dataset 2: Dados com previsões (2024 + 2026) - Para comparações
df_previsoes = dados_2026[['Pais', 'Produto', 'Valor_USD', 'Valor_Previsto_2026']].copy()
df_previsoes.rename(columns={'Valor_USD': 'Valor_2024', 'Valor_Previsto_2026': 'Valor_2026'}, inplace=True)

In [65]:
# Calcular crescimento percentual por país
crescimento_pais = df_previsoes.groupby('Pais').agg({
    'Valor_2024': 'sum',
    'Valor_2026': 'sum'
}).reset_index()
crescimento_pais['Crescimento_Percentual'] = ((crescimento_pais['Valor_2026'] - crescimento_pais['Valor_2024']) / crescimento_pais['Valor_2024'] * 100)

In [66]:
# Dataset 3: Evolução com previsão - Para gráfico temporal
evolucao_completa = []

In [67]:
# Dados históricos
for ano in range(2019, 2025):
    valor_ano = df[df['Ano'] == ano]['Valor_USD'].sum()
    evolucao_completa.append({
        'Ano': ano,
        'Valor_Total': valor_ano,
        'Tipo': 'Histórico'
    })

In [68]:
# Previsão 2026
valor_2026 = df_previsoes['Valor_2026'].sum()
evolucao_completa.append({
    'Ano': 2026,
    'Valor_Total': valor_2026,
    'Tipo': 'Previsão'
})

df_evolucao = pd.DataFrame(evolucao_completa)

In [69]:
# 8. EXPORTAR DADOS PARA LOOKER STUDIO
# Salvar datasets
df_historico.to_csv('dados_historicos_looker.csv', index=False, encoding='utf-8')
crescimento_pais.to_csv('crescimento_paises_2026.csv', index=False, encoding='utf-8')
df_evolucao.to_csv('evolucao_temporal_completa.csv', index=False, encoding='utf-8')
df_previsoes.to_csv('comparacao_2024_2026.csv', index=False, encoding='utf-8')

print("       Arquivos exportados:")
print("       dados_historicos_looker.csv")
print("       crescimento_paises_2026.csv")
print("       evolucao_temporal_completa.csv")
print("       comparacao_2024_2026.csv")

       Arquivos exportados:
       dados_historicos_looker.csv
       crescimento_paises_2026.csv
       evolucao_temporal_completa.csv
       comparacao_2024_2026.csv


In [70]:
# 9. RESUMO EXECUTIVO DA ANÁLISE
print(f"\n   DADOS PROCESSADOS:")
print(f"   • Registros originais: {len(df_exportacao):,}")
print(f"   • Registros após limpeza: {len(df):,}")
print(f"   • Período analisado: {df['Ano'].min()}-{df['Ano'].max()}")
print(f"   • Países únicos: {df['Pais'].nunique()}")
print(f"   • Produtos únicos: {df['Produto'].nunique()}")

print(f"\n   ANÁLISE FINANCEIRA:")
total_periodo = df['Valor_USD'].sum()
print(f"   • Valor total exportado (2019-2024): US$ {total_periodo:,.0f}")
print(f"   • Valor médio anual: US$ {total_periodo/6:,.0f}")
print(f"   • Maior ano: {evolucao_anual.idxmax()} (US$ {evolucao_anual.max():,.0f})")

print(f"\n   MACHINE LEARNING:")
print(f"   • Melhor modelo: {melhor_nome}")
print(f"   • Precisão (R²): {melhor_r2:.4f}")
print(f"   • Registros de treinamento: {len(df_ml_limpo):,}")

print(f"\n   PREVISÕES 2026:")
total_2024 = crescimento_pais['Valor_2024'].sum()
total_2026 = crescimento_pais['Valor_2026'].sum()
crescimento_total = ((total_2026 - total_2024) / total_2024) * 100
print(f"   • Total previsto 2026: US$ {total_2026:,.0f}")
print(f"   • Crescimento esperado: {crescimento_total:+.2f}%")


   DADOS PROCESSADOS:
   • Registros originais: 5,566
   • Registros após limpeza: 5,566
   • Período analisado: 2019-2024
   • Países únicos: 10
   • Produtos únicos: 97

   ANÁLISE FINANCEIRA:
   • Valor total exportado (2019-2024): US$ 1,053,349,216,026
   • Valor médio anual: US$ 175,558,202,671
   • Maior ano: 2023 (US$ 212,515,398,380)

   MACHINE LEARNING:
   • Melhor modelo: Decision Tree
   • Precisão (R²): 0.9763
   • Registros de treinamento: 3,655

   PREVISÕES 2026:
   • Total previsto 2026: US$ 237,020,592,395
   • Crescimento esperado: +17.10%


In [71]:
# Top 5 países com maior crescimento percentual
top_5_crescimento = crescimento_pais.nlargest(5, 'Crescimento_Percentual')
print(f"\n  TOP 5 PAÍSES COM MAIOR CRESCIMENTO PREVISTO:")
for i, row in enumerate(top_5_crescimento.iterrows(), 1):
    _, data = row
    print(f"   {i}. {data['Pais']}: {data['Crescimento_Percentual']:+.1f}%")

print(f"\n PRONTO PARA LOOKER STUDIO:")
print("    4 arquivos CSV exportados")
print("    Dados limpos e estruturados")
print("    Previsões ML incluídas")
print("    Métricas de crescimento calculadas")

print("\n PRÓXIMOS PASSOS:")
print("   1. Importar os CSVs no Looker Studio")
print("   2. Criar os dashboards conforme planejado")
print("   3. Configurar filtros interativos")
print("   4. Aplicar formatação e cores")

print("\n" + "="*60)


  TOP 5 PAÍSES COM MAIOR CRESCIMENTO PREVISTO:
   1. Argentina: +60.3%
   2. Países Baixos (Holanda): +37.3%
   3. México: +37.0%
   4. Japão: +30.2%
   5. Estados Unidos: +20.4%

 PRONTO PARA LOOKER STUDIO:
    4 arquivos CSV exportados
    Dados limpos e estruturados
    Previsões ML incluídas
    Métricas de crescimento calculadas

 PRÓXIMOS PASSOS:
   1. Importar os CSVs no Looker Studio
   2. Criar os dashboards conforme planejado
   3. Configurar filtros interativos
   4. Aplicar formatação e cores

